In [1]:
import numpy as np
import tensorflow as tf
import sklearn
import csv
import pandas as pd

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# MovieLens 100k

In [2]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', names=names, sep='\t')

In [3]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]

In [4]:
print(n_users)
print(n_items)

943
1682


In [5]:
nan = np.nan
movielens_ratings_matrix = np.zeros((n_users, n_items)) * nan
for line in df.itertuples():
    movielens_ratings_matrix[line[1]-1, line[2]-1] = line[3]

In [6]:
print(movielens_ratings_matrix)

[[ 5.  3.  4. ... nan nan nan]
 [ 4. nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [ 5. nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan  5. nan ... nan nan nan]]


# Introduction

In model-based methods, a summarized model of data is created up front, as with supervised and unsupervised learning methods. Therefore, the training is clearly separated from the prediction phase. <br>
Examples of such methods in traditional machine learning include decision trees, rule-based methods, Bayes classifiers, regression models, support vector machines, and neural networks.

Unlike data classification, any entry in the ratings matrix maybe missing.

# Decision and Regression Trees

Gini index lies between 0 and 1, with smaller value being more indicative of greater discriminative power: $$ G(S) = 1 - \sum_{i=1}^r p_i^2 $$

$$ Gini(S \Rightarrow [S_i, S_2] = \dfrac{n_1.G(S_1) + n_2.G(S_2)}{n_1 + n_2} $$

### Binary matrix

In [7]:
class BinaryMatrix():
    def __init__(self):
        pass
    
    def random_init(self, size):
        self.matrix = np.random.randint(2, size=size)
        
    def get_label(self):
        return self.matrix[:, -1]
    
    def get_train_data(self):
        return self.matrix[:, :-1]

In [8]:
binary_matrix = BinaryMatrix()
binary_matrix.random_init(size=[100, 100])

In [9]:
print(binary_matrix.matrix)
print(binary_matrix.get_label())
print(binary_matrix.get_train_data())

[[1 0 0 ... 1 1 1]
 [1 0 0 ... 1 1 1]
 [1 1 1 ... 0 1 1]
 ...
 [0 1 0 ... 1 0 0]
 [0 1 0 ... 0 1 0]
 [0 1 1 ... 0 0 1]]
[1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 1 1 1 0 1 1 1 0 0 0 1 1 1 0
 1 0 1 1 0 1 0 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 1 1 0 1 1 0 0 1 0
 0 0 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1]
[[1 0 0 ... 0 1 1]
 [1 0 0 ... 0 1 1]
 [1 1 1 ... 1 0 1]
 ...
 [0 1 0 ... 1 1 0]
 [0 1 0 ... 1 0 1]
 [0 1 1 ... 1 0 0]]


In [10]:
from sklearn import tree
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(binary_matrix.get_train_data(),
                                                   binary_matrix.get_label(), 
                                                    test_size=0.2, random_state=42)

In [12]:
clf = tree.DecisionTreeClassifier(random_state=42)

In [13]:
clf.fit(X=X_train, y=y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [14]:
predict_test = clf.predict(X_test)

In [15]:
accuracy = np.sum(y_test == predict_test) / len(y_test)
print(accuracy)

0.6


In [16]:
!pip install graphviz

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Sparse Matrix

In [17]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

In [18]:
ratings_matrix = sparse_random_matrix(1000, 1000, density=0.05, random_state=42)

We need to choose $j^{th}$ item to be target, and others $n - 1$ columns to be features

In [19]:
svd = TruncatedSVD(n_components=10, n_iter=10, random_state=42)


Example: $j^{th}$ column is the last column

In [20]:
X_data = ratings_matrix[:, :-1]
y_data = ratings_matrix[:, -1]

In [21]:
svd.fit(X_data)
print(svd.singular_values_)

[1.99437071 1.99214704 1.97315971 1.96586987 1.95468439 1.94150428
 1.9347887  1.93256985 1.92155791 1.90991485]


Then we use Decision tree on density matrix $m \times d$

In [22]:
reduction_ratings_matrix = svd.transform(X_data)

In [23]:
print(reduction_ratings_matrix)

[[-0.04078747  0.01740403 -0.02456857 ... -0.13284343  0.05258134
   0.10191544]
 [-0.01047693 -0.0370801  -0.08007284 ... -0.02028828 -0.05122547
  -0.00393312]
 [-0.00324058  0.01206909 -0.05224786 ...  0.06357679  0.03219327
  -0.10568864]
 ...
 [ 0.14103169  0.02498886  0.06968854 ...  0.05991687 -0.0404613
   0.09306438]
 [-0.04791901 -0.05253255 -0.0669215  ... -0.00280977 -0.0266462
   0.05423395]
 [ 0.02091143 -0.01567058 -0.03447271 ...  0.01500747  0.07855035
  -0.03599526]]


In [24]:
clf = tree.DecisionTreeRegressor()

clf.fit(reduction_ratings_matrix, y_data.todense())

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [25]:
print(clf.predict(reduction_ratings_matrix))
print(clf.predict(reduction_ratings_matrix).shape)

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.14142136  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 -0.14142136  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.14142136  0.          0.          0.
  0.14142136  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.14142136  0.
  0.          0.          0.          0.          0.          0.
  0.         -0.14142136 

We must loop through all items

# Rule-based Collaborative Filtering
(recommenderlab in R)

Consider a transaction database $ T = \{ T_1...T_m \} $ containing $m$ transactions, which are defined on $n$ items $I$. $I$ is the universal set of items, and each transaction $T_i$ is a subset of items in $I$.

$(\textbf{support})$ The $support$ of an item set $X \subseteq I$ is the fraction of transactions in $T$, of which $X$ is a subset <br>
If the support of an itemset is at least equal to predefined threshold $s$, then the itemset is said to be frequent. This threshold is referred to as the $minimum support$, these itemset are referred to as $frequent itemsets$ or $frequent patterns$

$(\textbf{Confidence})$ The confidence of the rule $X \Rightarrow Y$ is the conditional probability that a transaction in $T$ contains $Y$, given that it also contains $X$. Therefore, the confidence is obtained by dividing the support of $X \cup Y$ with the support of $X$

$(\textbf{Association Rules})$ A rule $X \Rightarrow Y$ is said to be an association rule at a minimum support of $s$ and minimum confidence of $c$, if the following two conditions are satisfied:<br>
1. The support of $X \cup Y$ is at least $s$
2. The confidence of $X \Rightarrow Y$ í at least $c$


# Naive  Bayes Collaborative Filtering

Bayes Rule: $$ P(A|B) = \dfrac{P(A).P(B|A)}{P(B)} $$

# Using an Arbitrary Classification Model as a Blackbox

The first step is to initialize the missing entries in the matrix with row averages, column averages, or with any simple collaborative filtering algorithm => remove bias, then fill 0 in the missing entries.

Using the following two steps iterative approach:
1. Use algorithm $A$ to estimate the missing entries of each column by setting it as the target variable and the remaining columns as the feature variables. For the remaining columns, use the current set of filled in values to create a complete matrix of feature variables. The observed ratings in the target column are used for training, the the missing ratings are predicted.
2. Update all the missing entries based on the prediction of algorithm $A$ on each target colum

#### Example

Firsly, we will substract the rating values by row averages, then fill 0 in the missing values 

In [26]:
def specified_rating_indices(u):
    return list(map(tuple, np.where(np.isfinite(u))))

In [27]:
# mean rating for each user i using his specified rating
def mean(u):
    specified_ratings = u[specified_rating_indices(u)]#u[np.isfinite(u)]
    m = sum(specified_ratings)/np.shape(specified_ratings)[0]
    return m

In [28]:
def all_user_mean_ratings(ratings_matrix):
    return np.array([mean(ratings_matrix[u, :]) for u in range(ratings_matrix.shape[0])])

In [29]:
def get_mean_centered_ratings_matrix(ratings_matrix):
    users_mean_rating = all_user_mean_ratings(ratings_matrix)
    mean_centered_ratings_matrix = ratings_matrix - np.reshape(users_mean_rating, [-1, 1])
    return mean_centered_ratings_matrix

In [30]:
mean_centered_ratings_matrix = get_mean_centered_ratings_matrix(movielens_ratings_matrix)

In [31]:
print(mean_centered_ratings_matrix)

[[ 1.38970588 -0.61029412  0.38970588 ...         nan         nan
          nan]
 [ 0.29032258         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]
 ...
 [ 0.95454545         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]
 [        nan  1.58928571         nan ...         nan         nan
          nan]]


In [32]:
def fill_zero_in_nan(matrix_2d):    
    for i in range(len(matrix_2d)):
        row = matrix_2d[i]
        
        for j in range(len(row)):
            if np.isnan(row[j]):
                matrix_2d[i][j] = 0
    
    return matrix_2d

In [33]:
row_average_rating_matrix = fill_zero_in_nan(mean_centered_ratings_matrix)

In [34]:
print(row_average_rating_matrix)

[[ 1.38970588 -0.61029412  0.38970588 ...  0.          0.
   0.        ]
 [ 0.29032258  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.95454545  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          1.58928571  0.         ...  0.          0.
   0.        ]]


Then we choose a column to be the target variable

To easily using neural network, we should find the item which is rated most

In [35]:
m = 0
m_index = 0

for i in range(n_items):
    y_i = movielens_ratings_matrix[:, i]
    total = 0

    for element in y_i:
        if not np.isnan(element):
            total = total + 1
    if total > m:
        m = total
        m_index = i
print(m)
print(m_index)

583
49


In [36]:
# for example, we will choose 
target_index = 49

X_data = row_average_rating_matrix[:, :target_index]
X_data = np.concatenate((X_data, row_average_rating_matrix[:, (target_index + 1):]), axis=1)

y_data = row_average_rating_matrix[:, target_index]
y_data_original = movielens_ratings_matrix[:, target_index]

In [37]:
train_indices = []
test_indices = []

for i in range(len(y_data_original)):
    if not np.isnan(y_data_original[i]):
        train_indices.append(i)
    else:
        test_indices.append(i)

In [38]:
X_train = X_data[train_indices]
y_train = y_data[train_indices]

X_test = X_data[test_indices]
y_test = y_data[test_indices]

In [39]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [40]:
print(X_train.shape)
print(X_val.shape)

(524, 1681)
(59, 1681)


In [41]:
from sklearn.utils import shuffle

def get_batch(X, y, iteration, batch_size):
    indices = range(iteration * batch_size, (iteration + 1) * batch_size)
    
    return X[indices], y[indices]

We will create a neural network to train:

In [42]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, n_items-1], name='X')
y = tf.placeholder(tf.float32, shape=None, name='y')

network = tf.layers.dense(inputs=X, activation=tf.nn.relu, units=256)
network = tf.layers.dense(inputs=network, activation=tf.nn.relu, units=128)
network = tf.layers.dense(inputs=network, activation=tf.nn.relu, units=128)
network = tf.layers.dense(inputs=network, activation=tf.nn.relu, units=128)
network = tf.layers.dense(inputs=network, activation=tf.nn.relu, units=32)

outputs = tf.layers.dense(inputs=network, units=1)

loss = tf.losses.mean_squared_error(labels=y, predictions=outputs)

train_op = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train_op = train_op.minimize(loss)

init = tf.global_variables_initializer()

In [43]:
n_epochs = 50
batch_size = 8

with tf.Session() as sess:
    sess.run(init)
    
    X_train, y_train = shuffle(X_train, y_train)
    
    for epoch in range(n_epochs):
        train_loss = 0
        
        for i in range(len(X_train) // batch_size):
            X_batch, y_batch = get_batch(X_train, y_train, iteration=i, batch_size=batch_size)

            _, loss_batch = sess.run([train_op, loss], feed_dict={X: X_batch, y: y_batch})

            train_loss = train_loss + loss_batch
    
        train_loss = train_loss / (len(X_train) // batch_size)
        val_loss = sess.run(loss, feed_dict={X: X_val, y: y_val})
        
        print("Epoch ", epoch, "\t Training loss: ", train_loss, "\t Validation loss: ", val_loss)
    
    y_predict = sess.run(outputs, feed_dict={X: X_test})

Epoch  0 	 Training loss:  1.077972957262626 	 Validation loss:  0.82471585
Epoch  1 	 Training loss:  0.8618305119184347 	 Validation loss:  0.73893553
Epoch  2 	 Training loss:  0.810829169016618 	 Validation loss:  0.72461456
Epoch  3 	 Training loss:  0.7979792961707481 	 Validation loss:  0.7211445
Epoch  4 	 Training loss:  0.791462045449477 	 Validation loss:  0.7187523
Epoch  5 	 Training loss:  0.7862932361089267 	 Validation loss:  0.7165719
Epoch  6 	 Training loss:  0.7817765217560988 	 Validation loss:  0.7145987
Epoch  7 	 Training loss:  0.7777542283901802 	 Validation loss:  0.7127913
Epoch  8 	 Training loss:  0.7741373626085428 	 Validation loss:  0.7111686
Epoch  9 	 Training loss:  0.7708963639461077 	 Validation loss:  0.7097496
Epoch  10 	 Training loss:  0.7679761471656653 	 Validation loss:  0.70847917
Epoch  11 	 Training loss:  0.7653097714369114 	 Validation loss:  0.70734936
Epoch  12 	 Training loss:  0.7628743907580009 	 Validation loss:  0.70635575
Epoch 

May be this result is not good as expected, but we can use this approach. <br>
We've predict ratings for one items, it is similar for other items.

# Latent Factor Models

## Low-Rank Intuition for Latent Factor Models

The rank-k ratings matrix $R$ with size $m \times n$ can always be expressed in the following product form of rank-k factors: $$ R = UV^T $$ <br>
Here $U$ is an $m \times k$ matrix, and $V$ is an $n \times k$ matrix

Even when the ratings matrix $R$ has rank larger than $k$ , it can be often approximately expressed as the product of rank-k factors: $$ R \approx UV^T $$ <br>
The error of this approximation is equal to $ || R - UV^T ||^2 $

# Basic Matrix Factorization Principles

In the basic matrix factorization model, the $m \times n$ ratings matrix $R$ is approximately factorized in to an $m \times k$ matrix $U$ and $n \times k$ matrix $V$, as follows: $$ R \approx UV^T $$ <br>
Each column of $U$ or $V$ is referred to as a $latent\ vector$ or $latent\ component$, where as each row of $U$ or $V$ is referred to as a $ latent\ factor $ <br>
Note: row $\bar{u_i}$ is a user factor, row $\bar{v_i}$ is a item factor, then: $$ r_{ij} \approx \bar{u_i}.\bar{v_i} $$

## Unconstrained Matrix Factorization

$$ Minimize\ J = \dfrac{1}{2} \|R - UV^T \|^2 $$
$$ subject\ to: $$
$$ No\ constrain\ on\ U\ and\ V $$ <br>
We only compute on observed entries

Residual Matrix: $ (R - UV^T) $

Let the set of all user-item pairs $(i, j)$, which are observed in R, be denoted by $S$: $$S = \{ (i, j):\ r_{ij}\ is\ observed \}$$

The $(i, j)$th entry of matrix R can be predicted as follows: $$ \hat{r}_{ij} = \sum_{s=1}^k u_{is}v_{js} $$

The difference between the observed and predicted value of a specified entri $(i, j)$ is given by: $$ e_{ij} = (r_{ij} - \hat{r}_{ij}) = (r_{ij} - \sum_{s=1}^k u_{is}v_{js}) $$<br>
The objective function now is: 
$$ Minimize\ J = \dfrac{1}{2} \sum_{(i, j) \in S} e_{ij}^2 = \dfrac{1}{2} \sum_{(i, j) \in S} (r_{ij} - \sum_{s=1}^k u_{is}v_{js})^2$$
$$ subject\ to: $$
$$ No\ constrain\ on\ U\ and\ V $$

In [83]:
def tf_specified_rating_indices(u):
    return tf.where(tf.is_finite(u))

In [88]:
def loss_on_observed_value(original_matrix, predicted_matrix):
    indices = tf_specified_rating_indices(original_matrix)
    
#     observed_values = original_matrix[indices]
#     predicted_value = predicted_matrix[indices]
    observed_values = tf.gather_nd(original_matrix, indices)
    predicted_values = tf.gather_nd(predicted_matrix, indices)
    
    loss = tf.losses.mean_squared_error(observed_values, predicted_values)
    
    return loss
    

In [110]:
k = 50

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[n_users, n_items], name='X')
# X = tf.Variable(initial_value=mean_centered_ratings_matrix, trainable=False)
# indices = tf.where(tf.is_finite(X))

U = tf.Variable(tf.random_uniform(shape=[n_users, k]))
V = tf.Variable(tf.random_uniform(shape=[n_items, k]))

outputs = tf.matmul(U, tf.transpose(V))

loss = loss_on_observed_value(X, outputs)

train_op = tf.train.GradientDescentOptimizer(learning_rate=1).minimize(loss)

init = tf.global_variables_initializer()

In [114]:
n_epochs = 5

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        _, train_loss = sess.run([train_op, loss], feed_dict={X: mean_centered_ratings_matrix})
        print("Epoch: ", epoch, "\t Loss: ", train_loss)
        
    u = sess.run(U)
    v = sess.run(V)
    print(np.matmul(u, np.transpose(v)))

Epoch:  0 	 Loss:  158.48889
Epoch:  1 	 Loss:  145.73935
Epoch:  2 	 Loss:  134.44907
Epoch:  3 	 Loss:  124.40249
Epoch:  4 	 Loss:  115.42244
[[10.399109  10.27696   10.839562  ... 11.085423  10.405196  10.95896  ]
 [11.851216  11.309194  10.718869  ... 11.789981  10.75323   12.846718 ]
 [10.354609   9.982286  10.064643  ...  9.698438  10.165601  10.766906 ]
 ...
 [ 9.9995575 10.037669  11.04409   ... 11.404249  10.263098  10.939229 ]
 [10.489732  10.336756   9.607583  ... 10.936442  10.561525  11.175063 ]
 [10.079789  10.320661   9.76344   ... 10.341753  10.096379  11.388627 ]]


May be overfit

### Regularization

$$ Minimize\ J = \dfrac{1}{2} \sum_{(i, j) \in S} e_{ij}^2 + \dfrac{\lambda}{2} \sum_{i = 1}^{m} \sum_{s = 1}^{k} u_{is}^2 + \dfrac{\lambda}{2} \sum_{j = 1}^{n} \sum_{s = 1}^{k} v_{js}^2 = \dfrac{1}{2} \sum_{(i, j) \in S} (r_{ij} - \sum_{s=1}^k u_{is}v_{js})^2 + \dfrac{\lambda}{2} \sum_{i = 1}^{m} \sum_{s = 1}^{k} u_{is}^2 + \dfrac{\lambda}{2} \sum_{j = 1}^{n} \sum_{s = 1}^{k} v_{js}^2$$



In [127]:
k = 50

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[n_users, n_items], name='X')
# X = tf.Variable(initial_value=mean_centered_ratings_matrix, trainable=False)
# indices = tf.where(tf.is_finite(X))

U = tf.Variable(tf.random_uniform(shape=[n_users, k]))
V = tf.Variable(tf.random_uniform(shape=[n_items, k]))

outputs = tf.matmul(U, tf.transpose(V))

loss = loss_on_observed_value(X, outputs)
reg_losses = tf.nn.l2_loss(U) + tf.nn.l2_loss(V)

loss = loss + 0.01*reg_losses
train_op = tf.train.GradientDescentOptimizer(learning_rate=1).minimize(loss)

init = tf.global_variables_initializer()



In [128]:
n_epochs = 50

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        _, train_loss = sess.run([train_op, loss], feed_dict={X: mean_centered_ratings_matrix})
        print("Epoch: ", epoch, "\t Loss: ", train_loss)
        
    u = sess.run(U)
    v = sess.run(V)
    print(np.matmul(u, np.transpose(v)))

Epoch:  0 	 Loss:  377.01175
Epoch:  1 	 Loss:  347.8236
Epoch:  2 	 Loss:  322.37454
Epoch:  3 	 Loss:  300.0182
Epoch:  4 	 Loss:  280.24548
Epoch:  5 	 Loss:  262.6488
Epoch:  6 	 Loss:  246.90015
Epoch:  7 	 Loss:  232.73224
Epoch:  8 	 Loss:  219.92514
Epoch:  9 	 Loss:  208.29678
Epoch:  10 	 Loss:  197.69565
Epoch:  11 	 Loss:  187.9937
Epoch:  12 	 Loss:  179.08385
Epoch:  13 	 Loss:  170.87387
Epoch:  14 	 Loss:  163.28552
Epoch:  15 	 Loss:  156.25148
Epoch:  16 	 Loss:  149.71349
Epoch:  17 	 Loss:  143.62119
Epoch:  18 	 Loss:  137.93059
Epoch:  19 	 Loss:  132.60312
Epoch:  20 	 Loss:  127.6052
Epoch:  21 	 Loss:  122.906815
Epoch:  22 	 Loss:  118.48176
Epoch:  23 	 Loss:  114.30677
Epoch:  24 	 Loss:  110.360954
Epoch:  25 	 Loss:  106.62583
Epoch:  26 	 Loss:  103.084724
Epoch:  27 	 Loss:  99.722824
Epoch:  28 	 Loss:  96.526726
Epoch:  29 	 Loss:  93.48416
Epoch:  30 	 Loss:  90.58441
Epoch:  31 	 Loss:  87.8175
Epoch:  32 	 Loss:  85.174286
Epoch:  33 	 Loss:  82.646

### Incremental Latent Component Training

We first perform the update for $u_{iq}$ and $v_{jq}$ only for $q = 1$. The approach repeatedly cycles through all the observed entries in S while performing the update for $q = 1$ until covergence is reached. Therefore, we can learn the first pair of columns $\bar{U_1}$ and $\bar{V_1}$. Then the outer product matrix $\bar{U_1} \bar{V_1}^T$ is subtracted from $R$ (for observed entries). Continue for $q = 2$ to $k$: 
$$ R \approx UV^T = \sum_{q = 1}^k \bar{U_q} \bar{V_q}^T $$

### Alternating Least Squares and Coordinate Descent

Alternatiing Least Squares: <br>
Iterative approach:
1. Keeping U fixed, optimize V
2. Keeping V fixed, optimize U 

The drawback of ALS is that it is not quite as efficient as SGD in large-scale settings with explicit ratings.

Coordinate Descent:<br>
Fixing a subset of variable. All entries in $U$ and $V$ are fixed except for a single entry (or coordinate) in one of two matrices, which will be optimized.

### Incorporating User and Item Biases

User biases: $o_i$ <br>
Item biases: $p_j$ <br>
The model learn this two variables. The predicted value if the rating entry $(i, j)$ given by:
$$ \hat{r_{ij}} = o_i + p_j + \sum_{s=1}^k u_{is}.v_{js} $$ <br>
Then the error $e_{ij}$ is given by: 
$$ e_{ij} = r_{ij} - \hat{r_{ij}} = r_{ij} - o_i - p_j - \sum_{s=1}^k u_{is}.v_{js}  $$ <br>
And the loss funtion of this type is given by:
$$ J = \dfrac{1}{2} \sum_{(i, j) \in S} e_{ij}^2 + \dfrac{\lambda}{2} \sum_{i = 1}^{m} \sum_{s = 1}^{k} u_{is}^2 + \dfrac{\lambda}{2} \sum_{j = 1}^{n} \sum_{s = 1}^{k} v_{js}^2 + \dfrac{\lambda}{2} \sum_{i = 1}^{m} o_i^2 + \dfrac{\lambda}{2} \sum_{j = 1}^{n} p_j^2 $$

In [139]:
def get_loss_with_bias(original_matrix, predicted_matrix, user_biases, item_biases):
    indices = tf_specified_rating_indices(original_matrix)
    
    predicted_matrix = tf.transpose(predicted_matrix) - user_biases
    predicted_matrix = tf.transpose(predicted_matrix)
    predicted_matrix = predicted_matrix - item_biases
    
    observed_values = tf.gather_nd(original_matrix, indices)
    predicted_values = tf.gather_nd(predicted_matrix, indices)
    
    loss = tf.losses.mean_squared_error(observed_values, predicted_values)
    
    return loss
    

In [140]:
k = 50

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[n_users, n_items], name='X')
# X = tf.Variable(initial_value=mean_centered_ratings_matrix, trainable=False)
# indices = tf.where(tf.is_finite(X))

U = tf.Variable(tf.random_uniform(shape=[n_users, k]))
V = tf.Variable(tf.random_uniform(shape=[n_items, k]))

user_biases = tf.Variable(tf.random_uniform(shape=[n_users]))
item_biases = tf.Variable(tf.random_uniform(shape=[n_items]))

outputs = tf.matmul(U, tf.transpose(V))

# fix here
loss = get_loss_with_bias(X, outputs, user_biases, item_biases)
reg_losses = tf.nn.l2_loss(U) + tf.nn.l2_loss(V) + tf.nn.l2_loss(user_biases) + tf.nn.l2_loss(item_biases)

loss = loss + 0.01*reg_losses
train_op = tf.train.GradientDescentOptimizer(learning_rate=1).minimize(loss)

init = tf.global_variables_initializer()




In [141]:
n_epochs = 50

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        _, train_loss = sess.run([train_op, loss], feed_dict={X: mean_centered_ratings_matrix})
        print("Epoch: ", epoch, "\t Loss: ", train_loss)
        
    u = sess.run(U)
    v = sess.run(V)
    print(np.matmul(u, np.transpose(v)))

Epoch:  0 	 Loss:  357.94336
Epoch:  1 	 Loss:  331.11768
Epoch:  2 	 Loss:  307.74307
Epoch:  3 	 Loss:  287.22437
Epoch:  4 	 Loss:  269.0896
Epoch:  5 	 Loss:  252.96286
Epoch:  6 	 Loss:  238.53928
Epoch:  7 	 Loss:  225.57095
Epoch:  8 	 Loss:  213.85413
Epoch:  9 	 Loss:  203.21967
Epoch:  10 	 Loss:  193.52666
Epoch:  11 	 Loss:  184.65709
Epoch:  12 	 Loss:  176.51068
Epoch:  13 	 Loss:  169.00241
Epoch:  14 	 Loss:  162.05998
Epoch:  15 	 Loss:  155.62096
Epoch:  16 	 Loss:  149.63173
Epoch:  17 	 Loss:  144.04549
Epoch:  18 	 Loss:  138.82193
Epoch:  19 	 Loss:  133.92581
Epoch:  20 	 Loss:  129.32579
Epoch:  21 	 Loss:  124.995
Epoch:  22 	 Loss:  120.90912
Epoch:  23 	 Loss:  117.04705
Epoch:  24 	 Loss:  113.389656
Epoch:  25 	 Loss:  109.9202
Epoch:  26 	 Loss:  106.62375
Epoch:  27 	 Loss:  103.48663
Epoch:  28 	 Loss:  100.49687
Epoch:  29 	 Loss:  97.64372
Epoch:  30 	 Loss:  94.91707
Epoch:  31 	 Loss:  92.30817
Epoch:  32 	 Loss:  89.80906
Epoch:  33 	 Loss:  87.4123

Instead of having separate variables $o_i$ and $p_j$ for users and items, we can increase the size of the factor matrices to incorporate these bias variables. We need to add two additional columns to each factor matrix $U$ and $V$, to create a larger factor matrices of size $m \times (k+2)$ and $n \times (k + 2)$. The last two columns of each factor matrix are special, because they correspond to the bias components. We have:
$$u_{i, k+1} = o_i$$
$$u_{i, k+2} = 1$$
$$v_{j, k+1} = 1$$
$$u_{j, k+2} = p_j$$

Then the loss funtion is given as follows:
$$ Minimize\ J = \dfrac{1}{2} \sum_{(i, j) \in S} (r_{ij} - \sum_{s=1}^{k+2} u_{is}v_{js})^2 + \dfrac{\lambda}{2} \sum_{s=1}^{k+2} (\sum_{i=1}^m u_{is}^2 + \sum_{j=1}^n v_{js}^2 )  $$
$$ subject\ to: $$
$$ (k+2)th\ column\ of\ U\ contains\ only\ 1s $$
$$ (k+1)th\ column\ of\ V\ contains\ only\ 1s $$